<a href="https://colab.research.google.com/github/Pavel-Zinkevich/english_folklore/blob/main/Parsing_wiki_preparing_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
!pip install openai mwclient mwparserfromhell tiktoken

In [ ]:
# imports
import mwclient  # библиотека для работы с MediaWiki API для загрузки примеров статей Википедии
import mwparserfromhell  # Парсер для MediaWiki
import openai  # будем использовать для токинизации
import pandas as pd  # В DataFrame будем хранить базу знаний и результат токинизации базы знаний
import re  # для вырезания ссылок <ref> из статей Википедии
import tiktoken  # для подсчета токенов

In [ ]:
name = "English folklore"

# Задаем категорию и англоязычную версию Википедии для поиска
CATEGORY_TITLE = f"Category:{name}"
WIKI_SITE = "en.wikipedia.org"

In [ ]:
# Соберем заголовки всех статей
def titles_from_category(category: mwclient.page.Page, max_depth: int) -> set[str]:
    """Возвращает множество заголовков статей категории и её подкатегорий, без самих подкатегорий."""
    titles = set()
    for cm in category.members():
        time.sleep(0.1)
        if isinstance(cm, mwclient.page.Page):
            if cm.name.startswith("Category:") and max_depth > 0:
                # рекурсивно обрабатываем подкатегорию
                titles.update(titles_from_category(site.pages[cm.name], max_depth - 1))
            elif not cm.name.startswith("Category:"):
                # добавляем только обычные статьи
                titles.add(cm.name)
    return titles



# Инициализация объекта MediaWiki
# WIKI_SITE ссылается на англоязычную часть Википедии
site = mwclient.Site(WIKI_SITE)

# Загрузка раздела заданной категории
category_page = site.pages[CATEGORY_TITLE]
# Получение множества всех заголовков категории с вложенностью на один уровень
titles = titles_from_category(category_page, max_depth=1)


print(f"Создано {len(titles)} заголовков статей в категории {CATEGORY_TITLE}.")

In [ ]:
# Задаем секции, которые будут отброшены при парсинге статей
SECTIONS_TO_IGNORE = [
    "See also",
    "References",
    "External links",
    "Further reading",
    "Footnotes",
    "Bibliography",
    "Sources",
    "Citations",
    "Literature",
    "Footnotes",
    "Notes and references",
    "Photo gallery",
    "Works cited",
    "Photos",
    "Gallery",
    "Notes",
    "References and sources",
    "References and notes",
]


In [ ]:
import time
from requests.exceptions import HTTPError
import mwclient

def all_subsections_from_title_with_retry(
    title: str,
    sections_to_ignore: set[str] = SECTIONS_TO_IGNORE,
    site_name: str = WIKI_SITE,
    max_retries: int = 3
) -> list[tuple[list[str], str]]:
    """
    Версия функции с повторными попытками при ошибках
    """
    for attempt in range(max_retries):
        try:
            time.sleep(1)  # Увеличьте задержку здесь
            site = mwclient.Site(site_name)
            page = site.pages[title]
            text = page.text()

            # Остальной код функции...
            parsed_text = mwparserfromhell.parse(text)
            headings = [str(h) for h in parsed_text.filter_headings()]
            if headings:
                summary_text = str(parsed_text).split(headings[0])[0]
            else:
                summary_text = str(parsed_text)

            results = [([title], summary_text)]
            for subsection in parsed_text.get_sections(levels=[2]):
                results.extend(
                    all_subsections_from_section(subsection, [title], sections_to_ignore)
                )
            return results

        except HTTPError as e:
            if e.response.status_code == 429:
                wait_time = 2 ** attempt  # Экспоненциальная backoff стратегия
                print(f"Ошибка 429, жду {wait_time} секунд перед повторной попыткой...")
                time.sleep(wait_time)
            else:
                raise e
    raise Exception(f"Не удалось получить данные для '{title}' после {max_retries} попыток")

# Используйте новую функцию в цикле
wikipedia_sections = []
for i, title in enumerate(titles):
    try:
        print(f"Обрабатываю {i+1}/{len(titles)}: {title}")
        wikipedia_sections.extend(all_subsections_from_title_with_retry(title))
        time.sleep(2)  # Увеличьте задержку между статьями
    except Exception as e:
        print(f"Ошибка при обработке '{title}': {e}")
        continue

In [ ]:
import json
import pickle
with open('wikipedia_sections.pkl', 'wb') as f:
    pickle.dump(wikipedia_sections, f)
# with open('wikipedia_sections.pkl', 'rb') as f:
#     wikipedia_sections_loaded = pickle.load(f)

In [ ]:
import re

def clean_section_advanced(section: tuple[list[str], str]) -> tuple[list[str], str]:
    titles, text = section

    # 1. Удаляем ссылки <ref>...</ref>
    text = re.sub(r"<ref.*?</ref>", "", text, flags=re.DOTALL)

    # 2. Убираем шаблоны {{...}}
    text = re.sub(r"\{\{.*?\}\}", "", text, flags=re.DOTALL)

    # 3. Убираем вики-ссылки [[...]] -> оставляем только текст
    text = re.sub(r"\[\[(?:[^|\]]*\|)?([^\]]+)\]\]", r"\1", text)

    # 4. Убираем заголовки ==Heading==
    text = re.sub(r"==+\s*(.*?)\s*==+", r"\1", text)

    # 5. Убираем остатки тройных и двойных кавычек '' и '''
    text = re.sub(r"'{2,3}", "", text)

    # 6. Убираем конструкции типа "| writer = " или "| composer = " в начале текста
    text = re.sub(r"\|\s*\w+\s*=\s*[^|}]*", "", text)

    # 7. Убираем двоеточия в начале строки (часто используются в текстах песен)
    text = re.sub(r"(?m)^:+", "", text)

    # 8. Преобразуем множественные пробелы и переносы строк в один пробел
    text = re.sub(r"\s+", " ", text)

    # 9. Убираем пробелы вначале и конце
    text = text.strip()

    return (titles, text)
original_num_sections = len(wikipedia_sections)
# Применяем ко всем секциям
wikipedia_sections = [clean_section_advanced(ws) for ws in wikipedia_sections]

# Фильтруем короткие секции
wikipedia_sections = [ws for ws in wikipedia_sections if len(ws[1]) >= 100]

print(f"Отфильтровано {original_num_sections-len(wikipedia_sections)} секций, осталось {len(wikipedia_sections)} секций.")


In [ ]:
GPT_MODEL = "gpt-3.5-turbo"  # only matters insofar as it selects which tokenizer to use

# Функция подсчета токенов
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Возвращает число токенов в строке."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Функция разделения строк
def halved_by_delimiter(string: str, delimiter: str = "\n") -> list[str, str]:
    """Разделяет строку надвое с помощью разделителя (delimiter), пытаясь сбалансировать токены с каждой стороны."""

    # Делим строку на части по разделителю, по умолчанию \n - перенос строки
    chunks = string.split(delimiter)
    if len(chunks) == 1:
        return [string, ""]  # разделитель не найден
    elif len(chunks) == 2:
        return chunks  # нет необходимости искать промежуточную точку
    else:
        # Считаем токены
        total_tokens = num_tokens(string)
        halfway = total_tokens // 2
        # Предварительное разделение по середине числа токенов
        best_diff = halfway
        # В цикле ищем какой из разделителей, будет ближе всего к best_diff
        for i, chunk in enumerate(chunks):
            left = delimiter.join(chunks[: i + 1])
            left_tokens = num_tokens(left)
            diff = abs(halfway - left_tokens)
            if diff >= best_diff:
                break
            else:
                best_diff = diff
        left = delimiter.join(chunks[:i])
        right = delimiter.join(chunks[i:])
        # Возвращаем левую и правую часть оптимально разделенной строки
        return [left, right]


# Функция обрезает строку до максимально разрешенного числа токенов
def truncated_string(
    string: str, # строка
    model: str, # модель
    max_tokens: int, # максимальное число разрешенных токенов
    print_warning: bool = True, # флаг вывода предупреждения
) -> str:
    """Обрезка строки до максимально разрешенного числа токенов."""
    encoding = tiktoken.encoding_for_model(model)
    encoded_string = encoding.encode(string)
    # Обрезаем строку и декодируем обратно
    truncated_string = encoding.decode(encoded_string[:max_tokens])
    if print_warning and len(encoded_string) > max_tokens:
        print(f"Предупреждение: Строка обрезана с {len(encoded_string)} токенов до {max_tokens} токенов.")
    # Усеченная строка
    return truncated_string

# Функция делит секции статьи на части по максимальному числу токенов
def split_strings_from_subsection(
    subsection: tuple[list[str], str], # секции
    max_tokens: int = 1000, # максимальное число токенов
    model: str = GPT_MODEL, # модель
    max_recursion: int = 5, # максимальное число рекурсий
) -> list[str]:
    """
    Разделяет секции на список из частей секций, в каждой части не более max_tokens.
    Каждая часть представляет собой кортеж родительских заголовков [H1, H2, ...] и текста (str).
    """
    titles, text = subsection
    string = "\n\n".join(titles + [text])
    num_tokens_in_string = num_tokens(string)
    # Если длина соответствует допустимой, то вернет строку
    if num_tokens_in_string <= max_tokens:
        return [string]
    # если в результате рекурсия не удалось разделить строку, то просто усечем ее по числу токенов
    elif max_recursion == 0:
        return [truncated_string(string, model=model, max_tokens=max_tokens)]
    # иначе разделим пополам и выполним рекурсию
    else:
        titles, text = subsection
        for delimiter in ["\n\n", "\n", ". "]: # Пробуем использовать разделители от большего к меньшему (разрыв, абзац, точка)
            left, right = halved_by_delimiter(text, delimiter=delimiter)
            if left == "" or right == "":
                # если какая-либо половина пуста, повторяем попытку с более простым разделителем
                continue
            else:
                # применим рекурсию на каждой половине
                results = []
                for half in [left, right]:
                    half_subsection = (titles, half)
                    half_strings = split_strings_from_subsection(
                        half_subsection,
                        max_tokens=max_tokens,
                        model=model,
                        max_recursion=max_recursion - 1, # уменьшаем максимальное число рекурсий
                    )
                    results.extend(half_strings)
                return results
    # иначе никакого разделения найдено не было, поэтому просто обрезаем строку (должно быть очень редко)
    return [truncated_string(string, model=model, max_tokens=max_tokens)]


In [ ]:
# Делим секции на части
MAX_TOKENS = 1600
wikipedia_strings = []
for section in wikipedia_sections:
    wikipedia_strings.extend(split_strings_from_subsection(section, max_tokens=MAX_TOKENS))

print(f"{len(wikipedia_sections)} секций Википедии поделены на {len(wikipedia_strings)} строк.")


In [ ]:
from openai import OpenAI
from google.colab import userdata  # ← Импортируем userdata из Colab

EMBEDDING_MODEL = "text-embedding-ada-002"

# Получаем API ключ из секретов Colab
api_key = userdata.get("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

def get_embedding(text, model="text-embedding-ada-002"):
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [ ]:
df = pd.DataFrame({"text": wikipedia_strings})

df['embedding'] = df.text.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

SAVE_PATH = f"./{name}_2025.csv"
# Сохранение результата
df.to_csv(SAVE_PATH, index=False)

In [ ]:
df.tail()